In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q trl
!pip install git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes==0.37.2
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-s596931r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-s596931r
  Resolved https://github.com/huggingface/peft.git to commit a0788a3f92c8220f68d2185aeef0266d6b725bfe
  Installing build dependencies ... don

In [8]:
# Mouting Google Drive to the current Colab session for accessing files stored in the Drive
drive.mount('/content/drive')

NameError: name 'drive' is not defined

#**Authentication**


In [ ]:
from huggingface_hub import login
login()

 # **Loading and Preparing the Model**

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
repo_id = "meta-llama/Llama-2–7b-chat-hf"
base_model = AutoModelForCausalLM.from_pretrained(
 repo_id,
 device_map='auto',
 load_in_8bit=True,
 trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
base_model.config.use_cache = False

OSError: Incorrect path_or_model_id: 'meta-llama/Llama-2–7b-chat-hf'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

# **Preprocessing and Preparing Datasets**

In [7]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="Ahmaric_dataamharic_3_all.csv")
print("Dataset loaded")

FileNotFoundError: Unable to find '/content/Ahmaric_dataamharic_3_all.csv'

# **Training the Model**

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
config = LoraConfig(
 r=8,
 lora_alpha=32,
 target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
 lora_dropout=0.05,
 bias="none",
 task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, config)
training_arguments = TrainingArguments(
 output_dir="Trainer_output",
 per_device_train_batch_size=1,
 gradient_accumulation_steps=4,
 logging_steps=20,
 learning_rate=2e-4,
 fp16=True,
 max_grad_norm=0.3,
 max_steps=300,
 warmup_ratio=0.03,
 group_by_length=True,
 lr_scheduler_type="constant",
)
trainer = SFTTrainer(
 model=model,
 train_dataset=dataset["train"],
 dataset_text_field="text",
 args=training_arguments,
 max_seq_length=512,
)
trainer.train()